In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time

In [3]:
from numpy.core.fromnumeric import shape
def load_data(path,batch_size,input_size):
    
    normalize = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ]) 
    transform_dict = {"src":  normalize}  
    # train_path=path+"/train"
    # test_path=path+"/test"
    data = datasets.ImageFolder(root=path,transform=transform_dict["src"])
    # transform_dict1 = {"test":  normalize} 
    # test1 = datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # transform_dict = {"test":  normalize}
    # test= datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # train_size = int((1- (test_split + val_split)) * len(data))
    # test_size = int((1 - (val_split)) * len(data)) - train_size
    # val_size = len(data) - train_size - test_size
    train_size=int(0.75*len(data))
    test_size=int(len(data)-train_size)
    train, test = td.random_split(data,[train_size,test_size])

    data_loader_train = td.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    data_loader_test = td.DataLoader(test,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    # data_loader_val = td.DataLoader(val,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    return data_loader_train, data_loader_test

In [4]:
data_loader_train,data_loader_test=load_data(r"C:\Users\Admin\Desktop\COMP 6721\Project\datasets\Dataset3",32,64)

In [5]:
data_loader_test.batch_size

32

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_loader_train,data_loader_test=load_data(r"/content/drive/MyDrive/Project/datasets/Dataset3",32,64)

In [6]:
img2=image.imread(r"C:\Users\Admin\Desktop\COMP 6721\Project\datasets\Dataset3\train\1.2.826.0.1.3680043.8.498.10144276773552601236124857108161469299-c.png")
img2.shape
#                   1.2.826.0.1.3680043.8.498.10299385524344582994601970314833752000-c.png")
# plt.imshow(img2)

(380, 512)

In [ ]:


class AlexNet(torch.nn.Module):

    def __init__(self, num_classes):
        super().__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(64, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        logits = self.classifier(x)
        return logits

In [10]:
import torch.optim as optim
net = AlexNet(22).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

NameError: name 'AlexNet' is not defined

In [ ]:
###### Define and run your training loop here #########
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
net.to(device)
acc_array = []
num_epochs = 30
total_steps = len(data_loader_train)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(data_loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
        # print("kjnfjnrnkrn",i)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print("second time",i)
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            accuracy = (correct / total) * 100
            acc_array.append(float(accuracy))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/30], Step [10/59], Loss: 0.6569, Accuracy: 65.62%
Epoch [1/30], Step [20/59], Loss: 0.6119, Accuracy: 71.88%
Epoch [1/30], Step [30/59], Loss: 0.7268, Accuracy: 56.25%
Epoch [1/30], Step [40/59], Loss: 0.5686, Accuracy: 75.00%
Epoch [1/30], Step [50/59], Loss: 0.6701, Accuracy: 62.50%
Epoch [2/30], Step [10/59], Loss: 0.6771, Accuracy: 59.38%
Epoch [2/30], Step [20/59], Loss: 0.5953, Accuracy: 71.88%
Epoch [2/30], Step [30/59], Loss: 0.5330, Accuracy: 81.25%
Epoch [2/30], Step [40/59], Loss: 0.6226, Accuracy: 68.75%
Epoch [2/30], Step [50/59], Loss: 0.5660, Accuracy: 78.12%
Epoch [3/30], Step [10/59], Loss: 0.5317, Accuracy: 87.50%
Epoch [3/30], Step [20/59], Loss: 0.4979, Accuracy: 84.38%
Epoch [3/30], Step [30/59], Loss: 0.6027, Accuracy: 71.88%
Epoch [3/30], Step [40/59], Loss: 0.5172, Accuracy: 81.25%
Epoch [3/30], Step [50/59], Loss: 0.6224, Accuracy: 68.75%
Epoch [4/30], Step [10/59], Loss: 0.5953, Accuracy: 71.88%
Epoch [4/30], Step [20/59], Loss: 0.5757,

Epoch [28/30], Step [40/59], Loss: 0.5976, Accuracy: 71.88%
Epoch [28/30], Step [50/59], Loss: 0.6636, Accuracy: 62.50%
Epoch [29/30], Step [10/59], Loss: 0.6899, Accuracy: 59.38%
Epoch [29/30], Step [20/59], Loss: 0.7076, Accuracy: 56.25%
Epoch [29/30], Step [30/59], Loss: 0.5734, Accuracy: 75.00%
Epoch [29/30], Step [40/59], Loss: 0.5738, Accuracy: 75.00%
Epoch [29/30], Step [50/59], Loss: 0.6463, Accuracy: 65.62%
Epoch [30/30], Step [10/59], Loss: 0.5160, Accuracy: 84.38%
Epoch [30/30], Step [20/59], Loss: 0.6673, Accuracy: 65.62%
Epoch [30/30], Step [30/59], Loss: 0.5812, Accuracy: 78.12%
Epoch [30/30], Step [40/59], Loss: 0.5881, Accuracy: 75.00%
Epoch [30/30], Step [50/59], Loss: 0.7067, Accuracy: 59.38%
######## Training Finished in 370.55488896369934 seconds ###########


In [ ]:
######## Write your code here #############
net.eval() 
count=0
with torch.no_grad(): 
    correct = 0
    total = 0
    for data in data_loader_test:
        # count+=1
        # print(count)
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))

Test Accuracy of the model on the 621 test images: 74.07407407407408 %


## Training the dataset with AlexNet fetching directly from Pytorch

In [7]:
import torch.optim as optim

import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

Using cache found in C:\Users\Admin/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [18]:
###### Define and run your training loop here #########
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 5
total_steps = len(data_loader_train)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(data_loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
#         print("kjnfjnrnkrn",i)\\
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("second time",i)
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/5], Step [10/59], Loss: 0.7234, Accuracy: 56.25%
Epoch [1/5], Step [20/59], Loss: 0.6576, Accuracy: 65.62%
Epoch [1/5], Step [30/59], Loss: 0.5519, Accuracy: 78.12%
Epoch [1/5], Step [40/59], Loss: 0.5413, Accuracy: 81.25%
Epoch [1/5], Step [50/59], Loss: 0.4948, Accuracy: 81.25%
Epoch [2/5], Step [10/59], Loss: 0.5302, Accuracy: 84.38%
Epoch [2/5], Step [20/59], Loss: 0.6495, Accuracy: 65.62%
Epoch [2/5], Step [30/59], Loss: 0.6460, Accuracy: 65.62%
Epoch [2/5], Step [40/59], Loss: 0.5276, Accuracy: 78.12%
Epoch [2/5], Step [50/59], Loss: 0.6355, Accuracy: 68.75%
Epoch [3/5], Step [10/59], Loss: 0.6450, Accuracy: 71.88%
Epoch [3/5], Step [20/59], Loss: 0.5975, Accuracy: 71.88%
Epoch [3/5], Step [30/59], Loss: 0.7153, Accuracy: 62.50%
Epoch [3/5], Step [40/59], Loss: 0.5646, Accuracy: 75.00%
Epoch [3/5], Step [50/59], Loss: 0.5960, Accuracy: 71.88%
Epoch [4/5], Step [10/59], Loss: 0.6639, Accuracy: 65.62%
Epoch [4/5], Step [20/59], Loss: 0.6883, Accuracy: 62.50%

In [19]:
model.eval() 
perds = []
target = []
with torch.no_grad(): 
    correct = 0
    total = 0
    for data in data_loader_test:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        perds.extend(predicted)
        target.extend(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))
    
# plt.plot(Accuracies)

Test Accuracy of the model on the 621 test images: 69.24315619967794 %


In [10]:
!pip install torchmetrics

In [21]:
from torchmetrics.functional import precision_recall
from torchmetrics import F1Score
from torchmetrics import ConfusionMatrix
perds1 = torch.stack(perds)
target1 = torch.stack(target)
f1 = F1Score(num_classes=4).to(device)
print(f1(perds1, target1))
print(precision_recall(perds1, target1, average='macro', num_classes=4))
confmat = ConfusionMatrix(num_classes=4).to(device)
confmat(perds1, target1)

tensor(0.6924, device='cuda:0')
(tensor(0.3462, device='cuda:0'), tensor(0.5000, device='cuda:0'))


tensor([[  0, 191,   0,   0],
        [  0, 430,   0,   0],
        [  0,   0,   0,   0],
        [  0,   0,   0,   0]], device='cuda:0')